# Module 4: Car Model

In this Module, we give some basic ingredients to model KITT as a dynamical system. This forms the
underlying model for a virtual model that emulates the real KITT, and you can use this to predict its
behavior.

The model has two parts: modeling the velocity, and modeling the direction. This directly relates to the two controls you have: velocity setting and steering angle. The overall system is a non-linear “non-
holonomic system”, meaning it is hard to directly apply your control knowledge to this system. But decoupling the system into velocity and direction is a good way to approximately linearize the system.

The idea of deriving a good model is that it improves your understanding of the system. Also, it allows to test your control algorithm first off-line on your model, and if proven right, going to the real KITT
should be less painful.

**Learning objectives** System modeling in action: Physical model for steering a car.

**Deliverable** As part of the mid-term report: A brief discussion on the assignments, and a tested system
model.

**Preparation** Make sure you have done the following before the start of the scheduled lab day:
– Reading this Module
– Homework Tasks 1 and 2 in these sections.

**Time duration** Three lab sessions, three homework sessions.

(@B till here)

## Velocity Model 

We start by driving on a straight line, and consider a simple model of longitudinal KITT dynamics, described by Newton’s second law. The car’s motion is influenced by the following three forces:

```{figure} KITTwind.jpg
---
height/width: 150px
name: car-figurewing
---
Indicate the forces and their directions
```

- Accelerating force Fa due to the torque delivered by the engine.
- Braking force Fb due to the brakes. This force clearly opposes the motion and decelerates the car.

*Unfortunately, KITT does not have a brake! You can stop by letting KITT roll to standstill, or for a*
*short period apply a negative force Fb. The difference with a real brake is that if you apply Fb for*
*too long, or were already stopped, the car will drive backwards.*
- Force Fd due to the air drag and other types of friction. This opposing force also decelerates the
car and depends on the velocity v as

$$
F_d = b |v| + c v^2 \,.
$$

The absolute value is needed to ensure the friction opposes the driving direction.

For the moment, assume the following car parameters (not guaranteed to be realistic!):
- Mass of the car: m = 5.6 kg
- Viscous friction coefficient: b = 5 $\text{Nm}^{-1} \text{s}$
- Air drag coefficient: c = 0.1 $\text{Nm}^{-2}\text{s}^{2}$
- Maximum acceleration force: Fa,max = 10 N
- Maximum braking force: Fb,max = 14 N


The car’s state parameters are longitudinal position z (not to confuse this with (x, y) later), and corre-
sponding velocity v = z′ and acceleration a = v′, all functions of time.

**Homework Task 1**

1. ndicate the forces and their directions in Figure 6.1.
2. Write the differential equation describing the longitudinal car dynamics.
3. Check if the air drag is relevant for our setting. Assuming it is not, you can easily solve the first
order differential equation in closed form. Derive this solution!
4. Make acceleration and deceleration plots, both position z and velocity v as function of time. Use a
small step size, e.g., 0.01 second. Set the initial velocity v(0) to 0 meters per second. Simulate the
model for 8 seconds using the maximum acceleration force Fa,max. Do the same for deceleration,
starting from an initial velocity v(0), and applying Fb,max.
Interpret the results. Do they correspond to what you would expect from an accelerating/decelerating
car?
*Hint:* You can also use Simulink to create such a model.

**Velocity Assignments**

Our aim is to take our theoretical model, and tune it to the real KITT, by estimating the relevant parameters (i.e., model identification). As it is a simple model, it is not going to be perfect. The dynamics of the
car are non-linear, e.g., it takes some extra force to make the car start driving. Also in practice battery
conditions do not remain constant, the car dynamics are then time-varying. We will try to ignore these
effects.
Further, the motor settings range from 135 till 165, with 150 being “neutral”; you can assume that 165
corresponds to $F_{max}$, and 135 to $−F_{m a x}$. So you will need a translation table from input to force, that
hopefully is linear.
The control input is the speed setting that we transmit to KITT. The system state is the position and
velocity (derivative of position). The observations are the distance of KITT to the wall, as measured with
the distance sensors. Measure elapsed time using a Python timer, see Section 2.6.2.

1. Make sure the batteries remain at a constant voltage during this experiment. Record the battery
voltage at each experiment.
2. Start/stop at different speed settings (applied during fixed time intervals) and generate the associ-
ated position-time curves.
3. Consider how to determine velocity from such plots.
Hint: this is not as easy as it seems. Obviously you think of finite differences, (z(t) − z(t − T ))/T .
This will give you the velocity not at time t but at an earlier time. Also, if you sample fast, the
inaccuracy of position has a great effect on the accuracy of the velocity estimate. If you sample
slow, you obtain velocity at a significantly earlier time.
4. How quickly do you reach a constant velocity? (This depends on the speed setting.)
5. If you solved the differential equations by hand (ignoring the drag, which is probably very small),
then you can try to fit the theoretical solution to your measurements. Can you find settings for the
Fmax and friction parameters such that you obtain a good match?
6. If you drive backwards, does the same model fit?

```{figure} steering1.png
---
height/width: 150px
name: car-figuresteering1
---
Approximaion of KITT's movement
```





## Steering Model 

If we steer a car by making the front wheels make an angle to the baseline, then the car will follow a
circle. The angle φ of the wheels will determine the radius R of the circle (if the angle is zero, then the
radius will become infinity: driving straight is a limit case). The dynamic model for driving on a line
from the previous section remains valid, except that it now describes the distance we drive on the circle.
Thus, the hard part at this point is to derive a relation between the angle φ and the radius R.
Consider KITT with distance L between both wheel axes. The rear axis of the car is located at [0; 0] and
the front axis at [L; 0] in a way that they are parallel to each other. The front wheels are turned an angle φ
relative to the positive x-axis. If the car drives at a speed v, then after a very small time ∆t, the rear axis
is located at

$$
\left[
\begin{array}{c}
v \cos(\phi) \Delta t \\
0
\end{array}
\right]
$$
and the front axis at
$$
\left[
\begin{array}{c}
L + v \cos (\phi) \Delta t \\
v \sin (\phi) \Delta t
\end{array}
\right]
$$
Mentioned in the KITT movement figure above.
The car was first parallel to the positive x-axis. Therefore, if θ denotes the angle of the car relative to the
positive x-axis,

$$
\theta(t) = 0
$$
and,
$$
\theta(t + \Delta t) = \arctan \left( \frac{v \sin (\phi) \Delta t}{L} \right)
$$

We can now evaluate the rate at which KITT turns by,


$$
\frac{d}{dt} \theta(t) = \lim_{\Delta t \to 0} \frac{\theta(t + \Delta t) - \theta(t)}{\Delta t} 
= \lim_{\Delta t \to 0} \frac{\arctan \left( \frac{v \sin (\phi) \Delta t}{L} \right)}{\Delta t}
= \lim_{\Delta t \to 0} \left( 1 + \frac{v^2 \sin(\phi) \Delta t^2}{L^2} \right)^{-1} \frac{v \sin (\phi)}{L}
= \frac{v \sin (\phi)}{L}.
$$

We have found a relationship between the angle of KITT’s wheels and the rate at which KITT turns.
Finally, instead of working with angles, it is often easier to work with unit direction vectors, e.g.,

$$
d(\alpha) = 
\left[
\begin{array}{c}
\cos(\alpha) \\
\sin(\alpha)
\end{array}
\right]
$$


where α is the orientation of the car (α = 0 if the car is parallel to the positive x-axis). The reason it is
easier is that we don’t have to worry about 2π wrapping if we increase or decrease α by some amount. If
we have a certain direction vector d and we want to rotate it over an angle θ, we can use a rotation matrix

$$
R(\theta) = 
\left[
\begin{array}{cc}
\cos(\theta) & -\sin(\theta) \\
\sin(\theta) & \cos(\theta)
\end{array}
\right]
$$


and set **d1 = R(θ)d0**. Similarly, we can define an orthogonal direction vector

$$
d(\alpha)^\perp = 
\left[
\begin{array}{c}
- \sin(\alpha) \\
\cos(\alpha)
\end{array}
\right]
$$

that points into the direction of the center of the driving circle (you may need to use −d⊥).
The state vector of the car consists of position x and orientation α, or equivalently unit direction vector
d(α), and a velocity v. You have to consider here what point of the car you consider to be the reference
point. The control input is the steering angle φ, or equivalently the direction setting given to KITT.

**Steering Assignments**

- Assume a constant velocity. Derive an equation for the radius R of the circle as function of steering
angle φ. Show the derivation in your report.
- Is this equation also valid in case the velocity is not constant?
- The wheelbase L is 33.5 cm (see Figure 3.1). Choose a direction setting and drive KITT in a circle.
Measure R and derive the corresponding steering angle φ.
Repeat for different direction settings and create a calibration table that relates the direction setting
to steering angle.
- Make a more detailed plot than Figure 6.2, showing also the center of the circle, c. Which part of
the car (back wheels, center, front wheels) actually rotates around c? If the car is at position x and
the direction vector of the car is d(α), then what are the coordinates of c?
*Hint:* Use $d(\alpha)^\perp$.
- You now have all the ingredients to create a dynamic model, given the initial state (x0, d0, v) and a
steering angle φ, compute the next state after a time ∆t.
Hint: You also have to define to what part of the car x refers to. Presumably, this is the location of
the beacon.

## Combined Model 

As the final step, combine the steering model with the earlier velocity model to also update v over the
same period ∆t.
We recommend to create a class KITTmodel, with a very similar interface as the earlier class KITT,
i.e., it accepts the same driving and steering commands. The virtual model can update the state parame-
ters accordingly, each time the state is requested and each time the speed or steering setting is changed.
You will need an internal timer to track how much time has elapsed since the previous time the state was
updated (i.e., ∆t, which is not a constant).
This is more advanced OOP, and tricky to debug due to the real-time aspects. Develop this first for just
the velocity part and a one-dimensional distance z.
Alternatively, you can opt for a simpler approach where you develop a simple script that carries out a
series of commands, and records the resulting positions.

### Mid-term assessment 4 and report

In the midterm assessment, the TA will give you a short series of driving/steering commands (e.g., M165,
D150 for 0.5 sec, D170 for 0.5 sec, stop), your model predicts the position, and this is compared to the
actual position of KITT after following the same series of commands.
In the midterm report, present the model, and present test results that shows the accuracy of this model,
starting from a known state. Summarize in a conclusion: over what time period can you predict the new
position of the car with an accuracy better than 30 cm?

### After the Midterm: GUI integration

After the midterm, you will work on a controller that takes you from A to B. Part of this is to have a GUI
where you show the field and the measured position of the car.
If you have a good car model, you can develop the controller and test it on this model. You can show the
resulting estimated positions of the car in the GUI. This is convenient for debugging, since your testing
time on the real field is very limited.
If your model is reliable, then at a later stage you can try to fuse location estimates from the beacon
signals with location predictions from the model. For optimal fusion, you need to know the accuracy
(variance) of each of the estimates, so you can take a weighted average. You can also use the model
predictions to detect and reject possible outliers from the beacon method.
You will probably find that the car behavior is not constant; e.g., the velocity response depends on the
battery status and also (slightly) on the steering setting. You could extend your model to take that into
account.
Further, to make the virtual model interface similar to the actual car interface, you would have to integrate
the Module 3 Localization scripts in such a way in the class KITT that you can request the current state
(position, orientation) from the real car just as you request it from the virtual model.

